In [ ]:
# %%
# Improved DCGAN Implementation for Color Images (RGB)

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from PIL import Image
import glob
import random
import pickle
from tqdm import tqdm
from PIL import ImageEnhance

# %%
# Constants - Adjusted for color images
MAX_IMAGES_PER_CLASS = 10000
RESULT_SAVE_DIR = "Vivian_128x128_results"
LATENT_DIM = 512
IMAGE_SIZE = 128  # Changed to 128 for target resolution
BATCH_SIZE = 8
EPOCHS_PER_CLASS = 400
DATASET_PATH = "vivian_nsfw_sfw_cropped_combined_faces"
TOLERANCE = 89 #img resize tolerance, if not close, center crop done

# %%
class SingleClassDataGenerator(keras.utils.Sequence):
    '''Improved data generator with better augmentation for color images'''
    
    def __init__(self, dataset_path, batch_size=32, max_images=1000, shuffle=True):
        self.dataset_path = dataset_path
        self.batch_size = batch_size
        self.max_images = max_images
        self.shuffle = shuffle
        
        print(f"Loading images from: {dataset_path}")
        self._build_file_list()
        
        if self.shuffle:
            random.shuffle(self.file_list)
        self.indexes = np.arange(len(self.file_list))
        
    def _build_file_list(self):
        """Build file list for images"""
        if not os.path.exists(self.dataset_path):
            raise ValueError(f"Directory not found: {self.dataset_path}")
        
        # Support multiple image formats
        image_files = []
        for ext in ['*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG']:
            image_files.extend(glob.glob(os.path.join(self.dataset_path, ext)))
        
        if len(image_files) > self.max_images:
            image_files = random.sample(image_files, self.max_images)
            
        self.file_list = image_files
        print(f"Found {len(self.file_list)} images")
        
    def __len__(self):
        return len(self.file_list) // self.batch_size
        
    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_files = [self.file_list[i] for i in batch_indexes]
        
        X = self._generate_batch(batch_files)
        return X
        
    def _generate_batch(self, batch_files):
        actual_batch_size = len(batch_files)
        X = np.zeros((actual_batch_size, IMAGE_SIZE, IMAGE_SIZE, 3))
    
        for i, original_img_path in enumerate(batch_files):
            
            # Start a loop to handle potential failures with substitute images
            img_path_to_process = original_img_path
            load_successful = False
            
            # Max attempts to prevent infinite loop if all files are corrupt
            max_attempts = len(batch_files) * 2 
            attempts = 0
    
            while not load_successful and attempts < max_attempts:
                attempts += 1
                try:
                    img = Image.open(img_path_to_process).convert('RGB')
                    w, h = img.size
                    
                    # --- Image Preprocessing and Augmentation ---
                    
                    # 1. Cropping/Resizing Logic (same as original)
                    if abs(w - IMAGE_SIZE) <= TOLERANCE and abs(h - IMAGE_SIZE) <= TOLERANCE:
                        img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
                    else:
                        short_side = min(w, h)
                        left = (w - short_side) // 2
                        top = (h - short_side) // 2
                        right = left + short_side
                        bottom = top + short_side
                        img = img.crop((left, top, right, bottom)).resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
    
                    # 2. Simpler augmentation (same as original)
                    if random.random() < 0.3:
                        img = img.transpose(Image.FLIP_LEFT_RIGHT)
                    # Subtle brightness/contrast adjustment
                    if random.random() < 0.3:
                        enhancer = ImageEnhance.Brightness(img)
                        img = enhancer.enhance(random.uniform(0.9, 1.1))
                    
                    if random.random() < 0.3:
                        enhancer = ImageEnhance.Contrast(img)
                        img = enhancer.enhance(random.uniform(0.9, 1.1))
    
                    # 3. Normalization (same as original)
                    img_array = np.array(img, dtype=np.float32) / 255.0
                    img_array = (img_array - 0.5) / 0.5  # Normalize to [-1, 1]
                    X[i] = img_array
                    
                    # If we reached here, loading and processing was successful
                    load_successful = True
    
                except Exception as e:
                    # Log the error
                    print(f"Error loading {img_path_to_process}. Replacing with a random file. Original file: {original_img_path}. Error: {e}")
                    
                    # Select a random image from the batch_files list as a substitute
                    img_path_to_process = random.choice(batch_files)
    
            # Final safety check: if all attempts failed, fill with black image
            if not load_successful:
                 print(f"CRITICAL: Failed to load a valid image for batch slot {i} after {max_attempts} attempts.")
                 X[i] = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3)) # Fallback to black image
                 
        return X
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.file_list)
            np.random.shuffle(self.indexes)  # Add this line
            
# class SelfAttention(layers.Layer):
#     """Custom Self-Attention Layer for DCGAN"""
    
#     def __init__(self, **kwargs):
#         super(SelfAttention, self).__init__(**kwargs)
    
#     def build(self, input_shape):
#         channels = input_shape[-1]
        
#         # Query, Key, Value projections
#         self.f_conv = layers.Conv2D(channels // 8, 1, name='query')
#         self.g_conv = layers.Conv2D(channels // 8, 1, name='key')
#         self.h_conv = layers.Conv2D(channels // 2, 1, name='value')
#         self.o_conv = layers.Conv2D(channels, 1, name='output')
        
#         # Learnable scale parameter
#         self.gamma = self.add_weight(
#             name='gamma',
#             shape=(),
#             initializer='zeros',
#             trainable=True
#         )
        
#         super(SelfAttention, self).build(input_shape)
    
#     def call(self, x):
#         batch_size = tf.shape(x)[0]
#         height = tf.shape(x)[1]
#         width = tf.shape(x)[2]
#         channels = x.shape[-1]
        
#         # Query, Key, Value projections
#         f = self.f_conv(x)  # [batch, h, w, c//8]
#         g = self.g_conv(x)  # [batch, h, w, c//8]
#         h = self.h_conv(x)  # [batch, h, w, c//2]
        
#         # Reshape for attention computation
#         f_flat = tf.reshape(f, [batch_size, -1, channels // 8])  # [batch, h*w, c//8]
#         g_flat = tf.reshape(g, [batch_size, -1, channels // 8])  # [batch, h*w, c//8]
#         h_flat = tf.reshape(h, [batch_size, -1, channels // 2])  # [batch, h*w, c//2]
        
#         # Attention map: [batch, h*w, h*w]
#         attention = tf.nn.softmax(tf.matmul(f_flat, g_flat, transpose_b=True))
        
#         # Apply attention
#         o = tf.matmul(attention, h_flat)  # [batch, h*w, c//2]
#         o = tf.reshape(o, [batch_size, height, width, channels // 2])
#         o = self.o_conv(o)
        
#         # Residual connection with learnable scale
#         return x + self.gamma * o
    
#     def get_config(self):
#         config = super(SelfAttention, self).get_config()
#         return config


# def build_generator():
#     '''Deep generator with residual blocks and self-attention for diverse anime faces'''

#     def residual_block(x, filters):
#         """Residual block for better gradient flow"""
#         shortcut = x
        
#         x = layers.Conv2D(filters, 3, padding='same', use_bias=False)(x)
#         x = layers.BatchNormalization(momentum=0.9)(x)
#         x = layers.ReLU()(x)
#         x = layers.Conv2D(filters, 3, padding='same', use_bias=False)(x)
#         x = layers.BatchNormalization(momentum=0.9)(x)
        
#         # If channel dimensions don't match, project shortcut
#         if shortcut.shape[-1] != filters:
#             shortcut = layers.Conv2D(filters, 1, padding='same', use_bias=False)(shortcut)
        
#         x = layers.Add()([x, shortcut])
#         x = layers.ReLU()(x)
#         return x

#     noise_input = layers.Input(shape=(LATENT_DIM,), name='noise_input')

#     # Initial dense projection - deeper starting point
#     x = layers.Dense(4 * 4 * 1024, use_bias=False)(noise_input)
#     x = layers.BatchNormalization(momentum=0.9)(x)
#     x = layers.ReLU()(x)
#     x = layers.Reshape((4, 4, 1024))(x)
    
#     # Add residual blocks at initial resolution
#     x = residual_block(x, 1024)
#     x = residual_block(x, 1024)

#     # 4x4 → 8x8, 512 filters
#     x = layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
#     x = layers.BatchNormalization(momentum=0.9)(x)
#     x = layers.ReLU()(x)
#     x = residual_block(x, 512)

#     # 8x8 → 16x16, 512 filters
#     x = layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
#     x = layers.BatchNormalization(momentum=0.9)(x)
#     x = layers.ReLU()(x)
#     x = residual_block(x, 512)
#     x = SelfAttention()(x)  # Add attention at 16x16

#     # 16x16 → 32x32, 256 filters
#     x = layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
#     x = layers.BatchNormalization(momentum=0.9)(x)
#     x = layers.ReLU()(x)
#     x = residual_block(x, 256)

#     # 32x32 → 64x64, 128 filters
#     x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
#     x = layers.BatchNormalization(momentum=0.9)(x)
#     x = layers.ReLU()(x)
#     x = residual_block(x, 128)
#     #x = SelfAttention()(x)  # Add attention at 64x64

#     # 64x64 → 128x128, 64 filters
#     x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
#     x = layers.BatchNormalization(momentum=0.9)(x)
#     x = layers.ReLU()(x)
#     x = residual_block(x, 64)
    
#     # Final output layer with tanh
#     generated_image = layers.Conv2D(3, kernel_size=7, padding='same', 
#                                    activation='tanh', use_bias=False, name='generated_image')(x)

#     generator = keras.Model(inputs=noise_input, outputs=generated_image, name='Generator')
#     return generator



# def build_discriminator():
#     '''Deep discriminator with residual blocks and self-attention'''
    
#     def residual_block(x, filters):
#         """Residual block for discriminator"""
#         shortcut = x
        
#         x = layers.Conv2D(filters, 3, padding='same')(x)
#         x = layers.LayerNormalization()(x)
#         x = layers.LeakyReLU(alpha=0.2)(x)
#         x = layers.Conv2D(filters, 3, padding='same')(x)
#         x = layers.LayerNormalization()(x)
        
#         if shortcut.shape[-1] != filters:
#             shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
        
#         x = layers.Add()([x, shortcut])
#         x = layers.LeakyReLU(alpha=0.2)(x)
#         return x

#     image_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='image_input')

#     # 128x128 → 64x64, 64 filters
#     x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(image_input)
#     x = layers.LeakyReLU(alpha=0.2)(x)
#     x = residual_block(x, 64)
#     #x = SelfAttention()(x)  # Attention at 64x64

#     # 64x64 → 32x32, 128 filters
#     x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
#     x = layers.LayerNormalization()(x)
#     x = layers.LeakyReLU(alpha=0.2)(x)
#     x = layers.Dropout(0.3)(x)
#     x = residual_block(x, 128)

#     # 32x32 → 16x16, 256 filters
#     x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
#     x = layers.LayerNormalization()(x)
#     x = layers.LeakyReLU(alpha=0.2)(x)
#     x = layers.Dropout(0.3)(x)
#     x = residual_block(x, 256)
#     x = SelfAttention()(x)  # Attention at 16x16

#     # 16x16 → 8x8, 512 filters
#     x = layers.Conv2D(512, kernel_size=4, strides=2, padding='same')(x)
#     x = layers.LayerNormalization()(x)
#     x = layers.LeakyReLU(alpha=0.2)(x)
#     x = layers.Dropout(0.3)(x)
#     x = residual_block(x, 512)

#     # 8x8 → 4x4, 1024 filters
#     x = layers.Conv2D(1024, kernel_size=4, strides=2, padding='same')(x)
#     x = layers.LayerNormalization()(x)
#     x = layers.LeakyReLU(alpha=0.2)(x)
#     x = layers.Dropout(0.3)(x)
#     x = residual_block(x, 1024)

#     # Final classification
#     x = layers.Flatten()(x)
#     x = layers.Dense(512)(x)
#     x = layers.LeakyReLU(alpha=0.2)(x)
#     x = layers.Dropout(0.3)(x)
#     validity = layers.Dense(1, activation='sigmoid', name='validity')(x)

#     discriminator = keras.Model(inputs=image_input, outputs=validity, name='Discriminator')
#     return discriminator

def build_generator():
    '''Improved generator with better architecture for 128x128 RGB output'''

    noise_input = layers.Input(shape=(LATENT_DIM,), name='noise_input')

    # Start with 4x4x1024 for 128x128 output
    x = layers.Dense(4 * 4 * 1024, use_bias=False)(noise_input)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)
    x = layers.Reshape((4, 4, 1024))(x)

    # 4x4 → 8x8, 512 filters
    x = layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    # 8x8 → 16x16, 256 filters
    x = layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    # 16x16 → 32x32, 128 filters
    x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    # 32x32 → 64x64, 64 filters
    x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.ReLU()(x)

    # 64x64 → 128x128, 3 filters (RGB output) - removed one layer
    generated_image = layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', 
                                           activation='tanh', use_bias=False, name='generated_image')(x)

    generator = keras.Model(inputs=noise_input, outputs=generated_image, name='Generator')
    return generator

def build_discriminator():
    '''Improved discriminator with dropout for 128x128 RGB input'''

    image_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='image_input')

    # 128x128 → 64x64, 64 filters
    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(image_input)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    # 64x64 → 32x32, 128 filters
    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    # 32x32 → 16x16, 256 filters
    x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    # 16x16 → 8x8, 512 filters
    x = layers.Conv2D(512, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    # 8x8 → 4x4, 1024 filters - removed one layer
    x = layers.Conv2D(1024, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(0.3)(x)

    # Flatten and output
    x = layers.Flatten()(x)
    validity = layers.Dense(1, activation='sigmoid', name='validity')(x)

    discriminator = keras.Model(inputs=image_input, outputs=validity, name='Discriminator')
    return discriminator

class ImprovedDCGAN:
    '''Improved DCGAN with better training dynamics'''
    
    def __init__(self, latent_dim=100):
        self.latent_dim = latent_dim
        
        # Build networks
        self.generator = build_generator()
        self.discriminator = build_discriminator()
        
        # Different learning rates for G and D
        self.gen_optimizer = Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.999)
        self.disc_optimizer = Adam(learning_rate=0.0004, beta_1=0.5, beta_2=0.999)
        
        # Loss function and metrics
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.1)
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
    
    def train_step(self, real_images):
        """Improved training step with better balance"""
        
        real_images = tf.convert_to_tensor(real_images, dtype=tf.float32)
        batch_size = tf.shape(real_images)[0]
        
        # Train discriminator
        with tf.GradientTape() as disc_tape:
            # Real images with label smoothing
            real_validity = self.discriminator(real_images, training=True)
            real_labels = tf.ones_like(real_validity) * 0.9
            real_loss = self.loss_fn(real_labels, real_validity)
            
            # Fake images
            noise = tf.random.normal([batch_size, self.latent_dim])
            generated_images = self.generator(noise, training=True)
            fake_validity = self.discriminator(generated_images, training=True)
            fake_labels = tf.zeros_like(fake_validity) + 0.1
            fake_loss = self.loss_fn(fake_labels, fake_validity)
            
            disc_loss = (real_loss + fake_loss) / 2
            
        disc_gradients = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        self.disc_optimizer.apply_gradients(zip(disc_gradients, self.discriminator.trainable_variables))
        
        # Train generator
        with tf.GradientTape() as gen_tape:
            noise = tf.random.normal([batch_size, self.latent_dim])
            generated_images = self.generator(noise, training=True)
            validity = self.discriminator(generated_images, training=True)
            
            # Generator wants discriminator to think images are real
            gen_loss = self.loss_fn(tf.ones_like(validity), validity)
            
        gen_gradients = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        self.gen_optimizer.apply_gradients(zip(gen_gradients, self.generator.trainable_variables))
        
        # Update metrics
        self.gen_loss_tracker.update_state(gen_loss)
        self.disc_loss_tracker.update_state(disc_loss)
        
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }
    
    def generate_images(self, num_samples=1):
        '''Generate images'''
        noise = tf.random.normal([num_samples, self.latent_dim])
        generated_images = self.generator(noise, training=False)
        
        # Convert from [-1, 1] to [0, 1] for display
        generated_images = (generated_images + 1) / 2.0
        
        return generated_images.numpy()
    
    def save_sample_images(self, epoch, num_samples=8):
        '''Save sample generated images'''
        
        generated_images = self.generate_images(num_samples)
        
        # Calculate grid dimensions dynamically
        cols = min(4, num_samples)
        rows = (num_samples + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
        
        # Handle single row/column cases
        if rows == 1 and cols == 1:
            axes = [axes]
        elif rows == 1 or cols == 1:
            axes = axes.flatten() if hasattr(axes, 'flatten') else [axes]
        else:
            axes = axes.flatten()
        
        for i in range(num_samples):
            axes[i].imshow(generated_images[i])
            axes[i].set_title(f'Generated Vivian')
            axes[i].axis('off')
        
        # Hide unused subplots
        for i in range(num_samples, len(axes)):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.savefig(f'{RESULT_SAVE_DIR}/new_generated_samples_vivian_epoch_{epoch}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Saved sample images for epoch {epoch}")

    def save_comparison_images(self, epoch, data_generator, num_samples=8):
        '''Save comparison between real and generated images'''
        
        # Get real images from data generator
        batch_idx = random.randint(0, len(data_generator) - 1)
        real_batch = data_generator[batch_idx]
        real_images = real_batch[:num_samples]
        
        # Generate fake images
        generated_images = self.generate_images(num_samples)
        
        # Create comparison plot
        fig, axes = plt.subplots(2, num_samples, figsize=(num_samples * 2, 4))
        
        for i in range(num_samples):
            # Real images (top row)
            real_img = (real_images[i] + 1) / 2.0  # Convert from [-1,1] to [0,1]
            axes[0, i].imshow(real_img)
            axes[0, i].set_title(f'Real Vivian' if i == 0 else '')
            axes[0, i].axis('off')
            
            # Generated images (bottom row)
            axes[1, i].imshow(generated_images[i])
            axes[1, i].set_title(f'Generated Vivian' if i == 0 else '')
            axes[1, i].axis('off')
        
        plt.tight_layout()
        plt.savefig(f'{RESULT_SAVE_DIR}/comparison_vivian_epoch_{epoch}.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Saved comparison images for epoch {epoch}")

# %%
def train_vivian():
    '''Improved training function with better monitoring'''
    
    print(f"\n{'='*60}")
    print(f"Training Vivian DCGAN")
    print(f"{'='*60}")
    
    # Initialize data generator
    data_generator = SingleClassDataGenerator(
        dataset_path=DATASET_PATH,
        batch_size=BATCH_SIZE,
        max_images=MAX_IMAGES_PER_CLASS,
        shuffle=True
    )
    
    if len(data_generator.file_list) == 0:
        print(f"No images found in {DATASET_PATH}, stopping...")
        return
    
    print(f"Training samples: {len(data_generator.file_list)}")
    print(f"Batches per epoch: {len(data_generator)}")
    
    # Initialize DCGAN
    dcgan = ImprovedDCGAN(LATENT_DIM)
    
    # Create directory for checkpoints
    checkpoint_dir = 'checkpoints_128x128/vivian'
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Track loss history for better monitoring
    g_losses = []
    d_losses = []
    
    # Training loop
    for epoch in range(EPOCHS_PER_CLASS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS_PER_CLASS}")
        
        # Reset metrics
        dcgan.gen_loss_tracker.reset_state()
        dcgan.disc_loss_tracker.reset_state()
        
        # Training step for each batch
        progress_bar = tqdm(range(len(data_generator)), desc=f"Epoch {epoch + 1}")
        for batch_idx in progress_bar:
            real_images = data_generator[batch_idx]
            losses = dcgan.train_step(real_images)
            
            progress_bar.set_postfix({
                'G_Loss': f"{losses['g_loss']:.4f}",
                'D_Loss': f"{losses['d_loss']:.4f}",
            })
        
        # Record losses
        avg_g_loss = dcgan.gen_loss_tracker.result().numpy()
        avg_d_loss = dcgan.disc_loss_tracker.result().numpy()
        g_losses.append(avg_g_loss)
        d_losses.append(avg_d_loss)
        
        print(f"Epoch {epoch + 1} completed - "
              f"Avg G Loss: {avg_g_loss:.4f}, Avg D Loss: {avg_d_loss:.4f}")
        
        # Save sample images every 5 epochs
        if (epoch + 1) % 5 == 0 or epoch==0:
            dcgan.save_sample_images(epoch + 1)

        if (epoch + 1) % 10 == 0:
            dcgan.save_comparison_images(epoch + 1, data_generator)
        
        # Save model checkpoints every 20 epochs
        if (epoch + 1) % 30 == 0:
            dcgan.generator.save(f'{checkpoint_dir}/generator2_epoch_{epoch+1:03d}.keras')
            dcgan.discriminator.save(f'{checkpoint_dir}/discriminator2_epoch_{epoch+1:03d}.keras')
            print(f"Models saved at epoch {epoch + 1}")
        
        # Improved early stopping conditions
        if avg_d_loss < 0.01:
            print("Discriminator loss too low, stopping training to prevent mode collapse")
            break
            
        if avg_g_loss > 6.0:
            print("Generator loss too high, stopping training")
            break
            
        # Check for loss explosion (sudden increase)
        if epoch > 5:
            recent_g_avg = np.mean(g_losses[-5:])
            if avg_g_loss > recent_g_avg * 2:
                print("Generator loss spiking, stopping training")
                break
        
        # Shuffle data
        data_generator.on_epoch_end()
    
    # Save final models with specified name
    dcgan.generator.save('vivianxsp2GenNet.keras')
    dcgan.discriminator.save(f'{checkpoint_dir}/final_discriminator.keras')
    print(f"Final generator saved as vivianxspGenNet.keras!")
    
    # Generate final sample images
    dcgan.save_sample_images("final", 9)
    
    return dcgan

# %%
def main():
    '''Main training function'''
    
    if not os.path.exists(DATASET_PATH):
        print("ERROR: Dataset directory not found!")
        print("Please ensure Vivian images are available at:", DATASET_PATH)
        return
    
    print("Starting Vivian DCGAN training...")
    os.makedirs('checkpoints', exist_ok=True)
    
    try:
        dcgan = train_vivian()
        print("\nTraining completed successfully!")
        print(f"Generator saved as: vivianxspGenNet.keras")
            
    except KeyboardInterrupt:
        print("\nTraining interrupted by user")
    except Exception as e:
        print(f"\nError during training: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

# %%

Starting Vivian DCGAN training...

Training Vivian DCGAN
Loading images from: vivian_nsfw_sfw_cropped_combined_faces
Found 5836 images
Training samples: 5836
Batches per epoch: 729

Epoch 1/400



poch 1: 100%|██████████████████████████████████████████| 729/729 [06:16<00:00,  1.94it/s, G_Loss=3.2270, D_Loss=0.6131]

Epoch 1 completed - Avg G Loss: 3.2270, Avg D Loss: 0.6131
Saved sample images for epoch 1

Epoch 2/400



poch 2: 100%|██████████████████████████████████████████| 729/729 [07:02<00:00,  1.72it/s, G_Loss=2.1034, D_Loss=0.5387]

Epoch 2 completed - Avg G Loss: 2.1034, Avg D Loss: 0.5387

Epoch 3/400


...


poch 189: 100%|████████████████████████████████████████| 729/729 [07:09<00:00,  1.70it/s, G_Loss=1.9151, D_Loss=0.4099]

Epoch 189 completed - Avg G Loss: 1.9151, Avg D Loss: 0.4099

Epoch 190/400



poch 190: 100%|████████████████████████████████████████| 729/729 [07:31<00:00,  1.61it/s, G_Loss=1.9141, D_Loss=0.4093]

Epoch 190 completed - Avg G Loss: 1.9141, Avg D Loss: 0.4093
Saved sample images for epoch 190
Saved comparison images for epoch 190

Epoch 191/400


Epoch 191:   4%|█▌                                       | 28/729 [00:17<07:42,  1.52it/s, G_Loss=1.9199, D_Loss=0.4077]